In [1]:
import glob
import os 
import scipy.io.wavfile as wav
from scipy.fftpack import fft
import numpy as np
folder_path = 'data/'
label = os.listdir(folder_path)
print(label)

music = {}
genre_x = []
genre_y = []
for idx, folder_name in enumerate(label):
    music[folder_name] = []
    for i in glob.glob(folder_path + str(folder_name) + '/*.wav'):
        rate, data = wav.read(i) # read .wav file
        # print(len(data))
        dataFFT = fft(data[0:67000])
        dataFFTAbs = abs(dataFFT[0:67000])

        music[folder_name].append(dataFFTAbs)
        genre_x.append(dataFFTAbs)
        genre_y.append(idx)
    print(folder_name, len(music[folder_name]))
print(len(genre_x), len(genre_y))

['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
blues 50
classical 50
country 50
disco 50
hiphop 50
jazz 50
metal 50
pop 50
reggae 50
rock 50
500 500


In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# print(np.array(genre_x).shape)
# train_x , test_x , train_y , test_y = train_test_split(genre_x , genre_y , test_size = 0.2 , random_state = 42)
train_x, train_y, test_x, test_y = [], [], [], []
for idx, values in enumerate(music.values()):
    # print(key, len(values[:40]))
    train_x = train_x + values[:40]
    train_y = train_y + [idx for num in range(len(values[:40]))]
    test_x  = test_x + values[40:]
    test_y  = test_y + [idx for num in range(len(values[40:]))]

print(len(train_x), len(train_y), len(test_x), len(test_y))


400 400 100 100


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(train_x, train_y)
Y = knn.predict(test_x)
print('Accuracy : ', accuracy_score(test_y , Y)*100)
confusion_matrix(test_y, Y)

# scores = cross_val_score(knn, genre_x, genre_y, cv=5, scoring='accuracy') # cv=num of fold
# print(scores)
# print(scores.mean())


Accuracy :  33.0


array([[0, 1, 2, 0, 0, 0, 6, 0, 1, 0],
       [0, 9, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 2, 0, 0, 3, 4, 0, 0, 0],
       [0, 1, 3, 2, 0, 0, 1, 0, 3, 0],
       [0, 1, 2, 0, 4, 1, 0, 0, 2, 0],
       [0, 4, 0, 0, 1, 3, 2, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 8, 0, 0, 0],
       [0, 0, 0, 3, 1, 2, 2, 2, 0, 0],
       [0, 1, 1, 0, 0, 1, 5, 0, 2, 0],
       [0, 1, 1, 0, 1, 0, 6, 0, 0, 1]], dtype=int64)

In [4]:
from xgboost import XGBClassifier

cls = XGBClassifier()

cls.fit(train_x , train_y)
Y = cls.predict(test_x)
print('accuracy : ' , accuracy_score(test_y , Y)*100)

# scores = cross_val_score(cls, genre_x, genre_y, cv=5, scoring='accuracy') # cv=num of fold
# print(scores)
# print(scores.mean())


accuracy :  27.0


In [5]:
from sklearn.svm import SVC

svm_clf = SVC(gamma='scale', decision_function_shape='ovo')
svm_clf.fit(train_x , train_y)
pred = svm_clf.predict(test_x)
print("Accuracy Score using Support Vector Machine Classifier = ", accuracy_score(test_y, pred)*100)
confusion_matrix(test_y, pred)


Accuracy Score using Support Vector Machine Classifier =  31.0


array([[1, 0, 0, 0, 0, 0, 3, 0, 1, 5],
       [1, 4, 3, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 2, 0, 0, 2, 4, 0, 0, 1],
       [0, 1, 1, 6, 1, 0, 0, 0, 0, 1],
       [0, 1, 1, 0, 4, 2, 1, 0, 1, 0],
       [2, 2, 2, 0, 1, 2, 0, 0, 0, 1],
       [0, 0, 1, 0, 4, 0, 5, 0, 0, 0],
       [0, 0, 0, 0, 5, 2, 0, 2, 0, 1],
       [0, 1, 1, 1, 0, 1, 1, 0, 3, 2],
       [0, 1, 1, 2, 0, 1, 3, 0, 0, 2]], dtype=int64)